In [ ]:
from cvxpy import *
import numpy as np
from math import sqrt
def unfold(X,n):
    ''' mode-n unfolding of tensor X
    Args: 
        X: input tensor
        n: targeted mode
    Returns:
        matricized version of X
    '''
    shape = np.shape(X)
    ndim = np.ndim(X)
    perm_order = np.roll(np.arange(ndim),n-1)
    X_n= np.reshape(np.transpose(X, perm_order), [shape[n-1],-1])
    return X_n

def norm_tensor(X, p=2):
    '''
    Wrapper function for tensor norm
    '''
    if p == 'nuc_upper':
        # tensor nuclear norm upper bound [FRIEDLAND && Lim 2014]
        expr_list = []
        X_shape = np.asarray(np.shape(X))
        for n in range(X.ndim):
            X_n = unfold(X, n)
            mask = np.ones(len(X_shape), dtype=bool)
            mask[n] = False
            print X_shape[mask]
            factor = sqrt(min(X_shape[mask]))
            norm_X_n = norm(X_n,'nuc')
            norm_X_n = norm_X_n * factor
            expr_list = expr_list + [norm_X_n]
        return min_elemwise(*expr_list)
        
    elif p == 'nuc_lower':
        # tensor nuclear norm lower bound 
        expr_list = []
        for n in range(X.ndim):
            X_n = unfold(X, n)
            expr_list = expr_list + [norm_X_n]
        return max_elemwise(expr_list)

In [ ]:
n1 = 20;
n2 = 30;
n3 = 10;
X = np.random.random((n1,n2,n3))
X_opt = Variable(n1, n2, n3)
Omega = np.random.choice([0,1], size = ((n1,n2,n3)), p = [0.8,0.2])

obj = Minimize(norm_tensor(X, 'nuc_upper'))
constraints = [mul_elemwise(Omega, X_opt) == mul_elemwise(Omega, X)]
prob = Problem(obj, constraints)
# Use SCS to solve the problem.
prob.solve(verbose=True, solver=SCS)